In [3]:

# Here we train and pickle a logistic regression model on the iris dataset

# Based on:
# http://scikit-learn.org/stable/auto_examples/linear_model/plot_iris_logistic.html

import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
import pickle

# import data
iris = datasets.load_iris()
X = iris.data[:, :2]  # we only take the first two features.
Y = iris.target

# declare model
logreg = LogisticRegression(C=1e5, solver='lbfgs', multi_class='multinomial')

# fit the data
logreg.fit(X, Y)

# pickle the data (wouldn't normally do this but want to plot prediction vs the train data)
trainXY = np.hstack((X,Y.reshape(-1,1)))
with open('data/trainXY.pkl', 'wb') as f:
    pickle.dump(trainXY, f)

# pickle the model
with open('data/model_logreg.pkl', 'wb') as f:
    pickle.dump(logreg, f)

In [4]:
%%writefile flask_ex6.py

from flask import Flask, request
import pickle
import numpy as np
import matplotlib.pyplot as plt
from io import BytesIO

# Initialize app
app = Flask(__name__)

# load the pickled model
with open('data/model_logreg.pkl', 'rb') as f:
    boa_model = pickle.load(f)

# load the pickled training data to display with prediction
with open('boa_model.pkl', 'rb') as f:
    boa = pickle.load(f)

with open('wells_model.pkl', 'rb') as f:
    wells = pickle.load(f)

with open('chase_model.pkl', 'rb') as f:
    chase = pickle.load(f)

with open('USB_model.pkl', 'rb') as f:
    USB = pickle.load(f)

with open('LD_model.pkl', 'rb') as f:
    LD = pickle.load(f)

with open('Fair_model.pkl', 'rb') as f:
    fair = pickle.load(f)

with open('cal_model.pkl', 'rb') as f:
    cal = pickle.load(f)

trainX = trainXY[:,:2]
trainY = trainXY[:,2]

# Home page with form on it to submit new data
@app.route('/')
def get_new_data():
    return '''
        <form action="/predict" method='POST'>
          Sepal length (4.0 - 9.0):<br>
          <input type="text" name="length"> 
          <br>
          Sepal width (1.0 - 5.0):<br>
          <input type="text" name="width"> 
          <br><br>
          <input type="submit" value="Submit for class prediction">
        </form>
        '''

@app.route('/predict', methods = ["GET", "POST"])
def predict():
    # request the text from the form 
    length = float(request.form['length'])
    width = float(request.form['width'])
    X_n = np.array([[length, width]])
    
    # predict on the new data
    Y_pred = model.predict(X_n)

    # for plotting 
    X_0 = trainX[trainY == 0] # class 0
    X_1 = trainX[trainY == 1] # class 1
    X_2 = trainX[trainY == 2] # class 2
    
    # color-coding prediction 
    if Y_pred[0] == 0:
        cp = 'b'
    elif Y_pred[0] == 1:
        cp = 'r'
    else:
        cp = 'g'

    if plt:
        plt.clf() # clears the figure when browser back arrow used to enter new data

    plt.scatter(X_0[:, 0], X_0[:, 1], c='b', edgecolors='k', label = 'class 0')
    plt.scatter(X_1[:, 0], X_1[:, 1], c='r', edgecolors='k', label = 'class 1')
    plt.scatter(X_2[:, 0], X_2[:, 1], c='g', edgecolors='k', label = 'class 2')
    plt.scatter(X_n[:, 0], X_n[:, 1], c=cp, edgecolors='k', marker = 'd', \
        s=100, label = 'prediction')
    plt.xlabel('Sepal length')
    plt.ylabel('Sepal width')
    plt.title('Prediction plotted with training data')
    plt.legend()
        
    image = BytesIO()
    plt.savefig(image)
    out = image.getvalue(), 200, {'Content-Type': 'image/png'}
    return out

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8080, debug=True)

Writing flask_ex6.py
